In [1]:
import pandas as pd
from math import pi, sin, cos, exp, tan, acos, isclose

# Dicinoário de Dados
COEFICIENTE_BASAL    => KCB\
COEFICIENTE_CULTURA  => KC\
COEFICIENTE_EVAPO    => KE\
EVAPO_CULTURA        => ETC\
TRANSP_POTENCIAL     => TRP\
EVAPO                => E\
ARMAZENAMENTO_AGUA   => ARM\
Capacidade de Campo  => TAR\
ARM2                 => ARM2\
EVAPO_REFERENCIA     => ET0\
EVAPO_REAL           => ETR\
TR                   => TR\
TA                   => TA\
DIAJULIANO           => DIAJULIANO\
TARMEDIO             => TAR/2 \
Indice de Satisfação => ISNA\
PLUVIOSIDADE         => P\


In [38]:
#Termo Aerodinâmico    
def termoAeroDinamico(params, urx, urn, tx, tn):
    tm   = (tx + tn)/2
    #Pressão de saturação do vapor d'água do ar - Equação de Tetens [kPa]
    es   = (0.6108* exp(17.27*tx/(237.3+tx))+ 0.6108* exp(17.27*tn/(237.3+tn)))/2
    #Pressão real do vapor d'água do ar [kPa]   
    ea   = (urn * 0.6108* exp(17.27*tx/(237.3+tx))+ urx * 0.6108* exp(17.27*tn/(237.3+tn)))/ 200
    #Declividade da curva de pressão de saturação [kPa/oC]
    S    = 4098 * 0.6108* exp(17.27*tm/(237.3+tm))/((237.3 + tm)**2)
    #Pressão atmosférica [kPa]
    Patm = 101.3*((293-0.0065*params.Z)/(293))**5.26
    #Calor latente de evaporação [MJ/kg]
    lamb = 0.665E-3*Patm
    
    return S, tm, es, ea, lamb

#Termo Radioativo
def saldoDeRadiacao(params, doy, qg, tx, tn, ea):
    #Radiação solar extraterrestre

    #Correção relativa Terra-Sol
    dr   = 1 + 0.033 * cos(2*pi/365*doy)
    #Declinação Solar 
    decl = 0.409 * sin((2*pi/365*doy)-1.39)
    #Angulo Horário
    ws   = acos(-(params.FI*pi/180)*tan(decl))
    #Radiação Solar Extraterrestre
    Qo   = 37.568*dr*((ws*sin(params.FI*pi/180)*sin(decl))+(cos(params.FI*pi/180)*cos(decl)*sin(ws)))

    #Balanço de radiação
    #Ondas curtas
    
    #Radiação solar para dia de céu sem nebulosidade
    Qso = (0.75 + 2E-5*params.Z)*Qo
    Qoc = 0.77 * qg
    
    #Ondas longas
    Qol    = 4.903E-9*((((tx + 273.16)**4)+ ((tn + 273.16)**4))/2)*(0.34-0.14*ea**0.5)*(1.35*(qg/Qso)-0.35)

    return Qoc - Qol

#Transpiração Potencial
def transpiracaoPotencial(params, u2, urn, aETo_PM, t):
    if t < params.L_INI:
        Kcb = params.KCB_INI
    elif t < params.L_INI + params.L_CRES:
        Kcb = params.KCB_INI+(t-params.L_INI)/params.L_CRES*(params.KCB_MID-params.KCB_INI)
    elif t < params.L_INI + params.L_CRES + params.L_MID:
        Kcb = params.KCB_MID
    else:
        Kcb = params.KCB_MID+(t-(params.L_INI+params.L_CRES+params.L_MID))/params.L_FIM*(params.KCB_FIM-params.KCB_MID)
    
    trp = Kcb * aETo_PM
    h = max((Kcb/params.KCB_MID)*params.HX,params.H)
    kcx = max(1.2+(0.04*(u2-2)-0.004*(urn-45))*(params.H/3)**0.3,Kcb+0.05)

    return Kcb, kcx , trp

#Balanço de água no solo
def evaporacao(params, p, Kcb, kcx, ETo_PM, De_f, t):
    #Camada subsuperfícial - Evaporação
    fc = max(abs(((Kcb-params.KCB_INI)/(kcx-params.KCB_INI)))**(1 + 0.5*params.H),0.01)
    
    if p > 0: fw = 1
    else: fw = params.FW_INI
    
    few = min(1-fc,fw)
    TAE = 1000*(params.CC-0.5*params.PM)*params.ZE
    
    if t == 1: De_i = TAE - params.AFE
    else: De_i = max(De_f-p, 0)
        
    if De_i < params.AFE: kr = 1
    else: kr = max((TAE - De_i)/(TAE - params.AFE), 0)
            
    ke = min(kr*(kcx-Kcb), few*kcx)
    E = ke * ETo_PM
    
    if t == 1:
        Dp = max(p, 0)
        De_f = min(De_i - p+(E/few) + Dp, TAE)
    else:
        Dp = max(p-De_f, 0)
        De_f = min(De_f-p+(E/few) + Dp, TAE)
        
    Kc = Kcb + ke
    ETc = Kc * ETo_PM
    return ETc, ke, De_f, De_i


def evapotranspiracaoReal(params, ap, aKcb, aETc, aKe, aETo_PM, De_i, Dr_f, t):
    if t <= (params.L_INI + params.L_CRES + params.L_MID):
        Zr = ((aKcb - params.KCB_INI)/(params.KCB_MID - params.KCB_INI))*(params.ZRX - params.ZRN) + params.ZRN
    else:
        Zr = params.ZRN
    
    TAR = 1000.*(params.CC - params.PM)*Zr
    AFR = params.F*TAR
    
    if t == 1:
        Dr_i = min( max((TAR-AFR)-ap+aETc, 0), TAR)
        if Dr_i < AFR:
            ks = 1
        else:
            ks = max( (TAR-Dr_i)/(TAR-AFR), 0)

        Dp = max( ap-aETc-De_i, 0 )
        kcr = aKe + ks * aKcb
        ETR = kcr * aETo_PM
        Tr = ks * aKcb *aETo_PM
        Dr_f = max( min((TAR-AFR)-ap+ETR+Dp, TAR), 0 )
    else:
        Dr_i = min( max(Dr_f-ap-aETc, 0), TAR)
        if Dr_i < AFR:
            ks = 1
        else:
            ks = max( (TAR-Dr_i)/(TAR-AFR), 0)
        
        Dp = max( ap-aETc-Dr_f, 0)
        kcr = aKe + ks*aKcb
        ETR = kcr * aETo_PM
        Tr = ks*aKcb*aETo_PM
        Dr_f = min(Dr_f-ap+ETR+Dp, TAR)
    return  TAR, Dr_f

In [30]:
#Precisa instalar xlrd ("pip install xlrd" ou "conda install xlrd")
dados = pd.read_csv("datasets/Dados _Meteorologicos_2021a2022.csv", sep=';')
params = pd.read_csv("datasets/Parametros_DadosProcessados.csv", sep=';').squeeze()
resultados = pd.read_csv("datasets/DadosProcessados.csv", sep=';')
intermediarios = pd.read_csv("datasets/datas.csv", sep=';')
#tx = temperatura maxima
#tn = temperatura minima
#urx = umidade maxima
#urn = umidade minima
#u2 = vento
#qg = radiação
#p = precipitacao
#arm = 
#teta = 
#p = -
#irrigacao = 0
#exp = 0
params

ID                                                 40
Z                                               370.0
FI                                             -11.51
L_INI                                            17.0
L_CRES                                           26.0
L_MID                                            33.0
L_FIM                                            35.0
KCB_INI                                           0.5
KCB_FIM                                           0.5
KCB_MID                                          1.15
HX                                                2.1
H                                                 0.0
FW_INI                                            1.0
CC                                               0.14
PM                                               0.04
ZE                                                0.1
ZRN                                               0.1
ZRX                                               0.3
AFE                         

In [16]:
intermediarios

,indice,doy,tx,tn,urx,urn,ea,lamb,es,S,tm,Qn,ET0,kcb,trp,h,kcx
0,1,44448,31.6,22.1,77.6,51.2,2.222092,0.066372,3.654220,0.207562,26.85,9.096267,4.378004,0.462095,2.023053,0.17834,1.194512
1,2,44449,36.3,19.5,77.6,27.8,1.719038,0.066372,4.153184,0.218967,27.90,12.012211,5.406680,0.462095,2.498399,0.17834,1.217494
2,3,44450,37.7,21.2,77.4,25.3,1.798891,0.066372,4.517883,0.236763,29.45,11.955160,5.696186,0.462095,2.632179,0.17834,1.223497
3,4,44451,37.0,21.1,77.5,33.1,2.008132,0.066372,4.388569,0.232058,29.05,12.901711,6.759113,0.462095,3.123351,0.17834,1.228986
4,5,44452,36.5,21.1,77.5,31.2,1.922175,0.066372,4.304126,0.229158,28.80,11.924541,6.040618,0.462095,2.791339,0.17834,1.223669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
116,117,44564,31.2,20.6,100.0,52.3,2.401532,0.066372,3.485276,0.197678,25.90,16.098119,5.570933,-1.789176,-9.967380,0.43000,1.181463
117,118,44565,31.8,22.6,98.2,61.5,2.792061,0.066372,3.721741,0.211307,27.20,14.581082,5.083817,-1.845010,-9.379692,0.43000,1.167616
118,119,44566,29.5,22.2,100.0,68.8,2.756439,0.066372,3.399610,0.197168,25.85,11.531337,3.810624,-1.900843,-7.243398,0.43000,1.142379
119,120,44567,31.4,21.9,100.0,62.7,2.754750,0.066372,3.611888,0.205447,26.65,13.695590,4.616013,-1.956677,-9.032046,0.43000,1.151535


In [45]:
aARM = []
De_f = 0
De_i = 0
Dr_f = 0
#for idx, (doy, tx, tn, urx, urn, u2, qg, ap, _, _, _, _, _) in dados.iterrows():
for idx, (_, _, doy, _, tx, tn, urx, urn, ap, u2, _, qg, _) in dados.iterrows():
    doy = intermediarios.doy[idx]
    # Termo Aerodinâmico    
    S, tm, es, ea, lamb = termoAeroDinamico(params, urx, urn, tx, tn) #conferidos

    Qn = saldoDeRadiacao(params, doy, qg, tx, tn, ea) #com erro
    print(Qn - intermediarios.Qn[idx])
    Qn = intermediarios.Qn[idx]
    
    #Evapotranspiração de referência [mm/d]
    aETo_PM = (0.408*S*(Qn)+lamb*900*u2*(es-ea)/(tm + 273))/(S+lamb*(1+0.34*u2)) #erro de aproximação
    aETo_PM = intermediarios.ET0[idx]
    
    aKcb, kcx, trp = transpiracaoPotencial(params, u2, urn, aETo_PM, idx+1) #com erro
    print(aKcb - intermediarios.kcb[idx], kcx - intermediarios.kcx[idx], trp - intermediarios.trp[idx])
    
    #não verificados
    aETc, aKe, De_f, De_i = evaporacao(params, ap, aKcb, kcx, aETo_PM, De_f, idx+1)
    aTAR, Dr_f = evapotranspiracaoReal(params, ap, aKcb, aETc, aKe, aETo_PM, De_i, Dr_f,idx+1)
    aARM.append( max(aTAR - Dr_f,0) )

0.05474238641586204
0.03790510000000008 0.005488430105762276 0.1659486860161059
0.09966262428381079
0.03790510000000008 -0.017494370962118255 0.20494075504308817
0.09839033253533458
0.03790510000000008 -0.023497341390296134 0.21591451688973207
0.09712769740216487
0.03790510000000008 -0.028985771496058632 0.2562048375988528
0.09187081566244792
0.03790510000000008 -0.02366885483110126 0.22897023921444548
0.08863080442232096
0.03790510000000008 -0.01629377687648237 0.21662313225511687
0.08498658752471577
0.03790510000000008 -0.015264696231652275 0.2158522123788149
0.1074668062526154
0.03790510000000008 -0.022982801067880754 0.2346189258544742
0.1123008041180853
0.03790510000000008 -0.03481722848343116 0.24971703172755344
0.12095363354828059
0.03790510000000008 -0.03687538977309246 0.24958020102413903
0.11858462001723957
0.03790510000000008 -0.04219230643804983 0.24189776930104046
0.1289660473808958
0.03790510000000008 -0.04425046772771091 0.2527271631823158
0.11826671600442218
0.037905100

In [54]:
#with pd.set_option('display.max_columns', None):
resultados.head()

,ID,FK_PARAMETRO,CHAVE_IMPORTACAO,CHAVE_PROCESSAMENTO,COEFICIENTE_CULTURA,COEFICIENTE_BASAL,COEFICIENTE_EVAPO,EVAPO_CULTURA,TRANSP_POTENCIAL,EVAPO,ARMAZENAMENTO_AGUA,TAR,ARM2,EVAPO_REFERENCIA,EVAPO_REAL,TR,TA,DIAJULIANO,TARMEDIO,STRESS,EVAPO_ACUMULADA,ISNA,FK_USUARIO,DATANEW,IRRIGACAO_EFETIVA,IRRIGACAO_PROJETADA,PLUVIOSIDADE,DEFICIT,EXCEDENTE,FK_PROPRIEDADE,INDICE,DATA_IMPORTACAO,DATA
0,73229,40,a9af814529a859ca71d125e37c902ca3,NaN,1.385883,0.5,0.885883,14.311259,5.163229,9.14803,0.0,10.0,0.0,10.326457,9.14803,0.0,0.0,249,5.0,NaN,NaN,0.639219,10,06/09/2021,0.0,10.0,0.0,10.0,0.0,1,1,2022-09-20 14:30:00,NaN
1,73230,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,4.725286,4.725286,0.00000,0.0,10.0,0.0,9.450572,0.00000,0.0,0.0,250,5.0,NaN,NaN,0.000000,10,07/09/2021,0.0,10.0,0.0,10.0,0.0,1,2,2022-09-20 14:30:00,NaN
2,73231,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,4.926514,4.926514,0.00000,0.0,10.0,0.0,9.853029,0.00000,0.0,0.0,251,5.0,NaN,NaN,0.000000,10,08/09/2021,0.0,10.0,0.0,10.0,0.0,1,3,2022-09-20 14:30:00,NaN
3,73232,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,3.103985,3.103985,0.00000,0.0,10.0,0.0,6.207971,0.00000,0.0,0.0,252,5.0,NaN,NaN,0.000000,10,09/09/2021,0.0,10.0,0.0,10.0,0.0,1,4,2022-09-20 14:30:00,NaN
4,73233,40,a9af814529a859ca71d125e37c902ca3,NaN,0.500000,0.5,0.000000,4.754294,4.754294,0.00000,0.0,10.0,0.0,9.508587,0.00000,0.0,0.0,253,5.0,NaN,NaN,0.000000,10,10/09/2021,0.0,10.0,0.0,10.0,0.0,1,5,2022-09-20 14:30:00,NaN


In [52]:
aETo_PM

13.285857228495745